In [1]:
import os
import re
import numpy as np

In [2]:
logfile_path = '../../ISWC2022LearnableFilter/Run_logs/logs/'

In [3]:
experiments = os.listdir(logfile_path)

# average over subjects

In [4]:
for e in experiments:
    experiment_path = logfile_path + e
    score_path = experiment_path + '/finetuned_score.txt'
    if not os.path.exists(score_path):
        score_path = experiment_path + '/score.txt'
    if os.path.exists(score_path):
        print(f'Processing {e}...')
        
        weighted_F1 = []
        macro_F1 = []
        
        f = open(score_path)
        while True:
            line = f.readline()
            if line == '':
                break
            scores = re.findall(r'0.[\d.]*\d+', line)
            weighted_F1.append(float(scores[0]))
            macro_F1.append(float(scores[1]))
        
        F1 = np.array([weighted_F1, macro_F1]).T
        mean_F1 = np.mean(F1, axis=0)
        np.savetxt(experiment_path+'/average_score.txt', mean_F1, fmt='%.6f')

Processing sahar_data_skodar_seed_2_windowsize_84_waveFilter_False_Fscaling_0.25_cvfilter_128_grufilter_None_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_skodar_seed_3_windowsize_84_waveFilter_False_Fscaling_0.25_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_dsads_seed_5_windowsize_125_waveFilter_False_Fscaling_0.5_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_skodar_seed_4_windowsize_84_waveFilter_False_Fscaling_0.25_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing sahar_data_oppo_seed_4_windowsize_30_waveFilter_False_Fscaling_1_cvfilter_128_grufilter_None_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_dsads_seed_2_windowsize_125_waveFilter_False_Fscaling_1_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_dsads_seed_4_windowsize_125_waveFilter_False_Fscaling_0.75_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_

/opt/bwhpc/common/jupyter/base/2022-03-30/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/bwhpc/common/jupyter/base/2022-03-30/lib/python3.8/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


Processing deepconvlstm_data_pamap2_seed_3_windowsize_168_waveFilter_False_Fscaling_0.75_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_dsads_seed_2_windowsize_125_waveFilter_False_Fscaling_0.5_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_oppo_seed_1_windowsize_30_waveFilter_False_Fscaling_0.75_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_skodar_seed_1_windowsize_84_waveFilter_False_Fscaling_0.75_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing sahar_data_oppo_seed_5_windowsize_30_waveFilter_False_Fscaling_0.25_cvfilter_128_grufilter_None_Regu_False_wavelearnble_None...
Processing deepconvlstm_data_oppo_seed_5_windowsize_30_waveFilter_False_Fscaling_0.75_cvfilter_64_lstmfilter_128_Regu_False_wavelearnble_None...
Processing sahar_data_oppo_seed_3_windowsize_30_waveFilter_False_Fscaling_1_cvfilter_128_grufilter_None_Regu_False_wavelearnble_No

# Average over seeds

In [5]:
# models = ['deepconvlstm', 'attend', 'sahar']
# datasets = ['oppo', 'hapt', 'skodar', 'pamap2', 'dsads']
# wavelets = ['False', 'True', 'None']
# sfs = ['1']

In [8]:
models = ['deepconvlstm']
datasets = ['pamap2','skodar','oppo', 'dsads']
wavelets = ['None']
sfs = ['0.25', '0.5', '0.75', '1']

In [9]:
for model in models:
    for dataset in datasets:
        for wavelet in wavelets:
            for sf in sfs:
                score_cross_seed = []
                for e in experiments:
                    if e.endswith('e'):
                        this_model = re.findall(r'(.+?)_data', e)[0]
                        this_scaling = re.findall(r'Fscaling_(.+?)_cvfilter', e)[0]
                        this_dataset = re.findall(r'_data_(.+?)_seed', e)[0]
                        this_wavelet = re.findall(r'wavelearnble_(.+)', e)[0]
            
                        if (this_model==model) and (this_dataset==dataset) and (this_scaling==sf) and (this_wavelet==wavelet):              
                            seed_path = logfile_path + e + '/average_score.txt'
                            score = np.loadtxt(seed_path)
                            score_cross_seed.append(score)
                print(score_cross_seed)
                m = np.mean(score_cross_seed, axis=0)
                std = np.std(score_cross_seed, axis=0)
                result_cross_seed = np.array([m, std])
                np.savetxt(f'{logfile_path}/../result/{model}_{dataset}_{sf}_{wavelet}.txt', result_cross_seed, fmt='%.6f')

[array([0.869243, 0.733819]), array([0.86581 , 0.744765]), array([0.880405, 0.763476]), array([0.875617, 0.748555]), array([0.844236, 0.742463])]
[array([0.864609, 0.738095]), array([0.855065, 0.741728]), array([0.813186, 0.736026]), array([0.873713, 0.767228]), array([0.81184 , 0.730246])]
[array([0.847968, 0.753425]), array([0.850678, 0.734868]), array([0.850436, 0.751266]), array([0.844454, 0.737818]), array([0.788289, 0.716714])]
[array([0.842058, 0.728708]), array([0.836053, 0.728144]), array([0.818406, 0.733084]), array([0.863316, 0.746201]), array([0.838377, 0.737218])]
[array([0.977565, 0.968879]), array([0.982376, 0.974792]), array([0.981912, 0.976333]), array([0.967108, 0.949761]), array([0.972614, 0.959023])]
[array([0.987317, 0.983067]), array([0.986183, 0.981179]), array([0.988483, 0.985383]), array([0.982127, 0.975334]), array([0.983414, 0.977962])]
[array([0.985283, 0.980399]), array([0.990269, 0.98776 ]), array([0.991859, 0.990932]), array([0.988247, 0.986334]), array([